<a href="https://colab.research.google.com/github/TapasKumarDutta1/Multihead_attention/blob/master/c4_c2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

from tensorflow.keras.activations import *

from tensorflow.keras.layers import *
from tensorflow.keras import backend as K
from tensorflow.keras.applications import *


def self_attention(inp,st):
    shp=inp.shape
    a=Conv2D(shp[3]//8,1, padding='same',kernel_initializer='he_uniform') (inp)
    a=Activation('relu') (a)
    b=Conv2D(shp[3]//8,1, padding='same',kernel_initializer='he_uniform') (inp)
    b=Activation('relu') (b)
    c=Conv2D(shp[3]//8,1, padding='same',kernel_initializer='he_uniform') (inp)
    c=Activation('relu') (c)
    a=GlobalAveragePooling2D()(a)
    a=Reshape(( 1, shp[3]//8),name='a'+st)(a)
    b=Reshape(( shp[1]*shp[2], shp[3]//8),name='b'+st)(b)
    b=K.permute_dimensions(b, (0, 2, 1))
    c=Reshape(( shp[1]*shp[2], shp[3]//8),name='c'+st)(c)
    inter=K.batch_dot(a,b)
    inter=Activation('softmax') (inter)
    out=K.batch_dot(inter,c)
    out=Reshape(( 1,1, shp[3]//8),name='d'+st)(out)
    out=Conv2D(shp[3],1, padding='same') (out)
    out=Activation('sigmoid') (out)
    out=out*inp
    out=Reshape(( shp[1],shp[2], shp[3]),name='e'+st)(out)
    
    
    inter=Reshape(( shp[1],shp[2], 1),name='f'+st)(inter)
    out1=inter*inp
    return Concatenate()([out,out1])

def spatial_attention(C_A):
    
    x=Lambda(lambda x: K.max(x,axis=-1,keepdims=True))  (C_A)
    x=Activation('relu') (x)
    x=Conv2D(1,1, padding='same') (x)
    x=Activation('sigmoid') (x)
    S_A=Multiply()([x,C_A])
    return S_A
class MyLayer(Layer):

    def __init__(self, **kwargs):
        super(MyLayer, self).__init__(**kwargs)

    def build(self, input_shape):

        self._x = K.variable(0.5)
        self._trainable_weights = [self._x]

        super(MyLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, x):
        A, B = x
        result = add([self._x*A ,(1-self._x)*B])
        return result

    def compute_output_shape(self, input_shape):
        return input_shape[0]
    
    
    
class GeM(Layer):
    def __init__(self, init_norm=3.0, normalize=False, **kwargs):
        self.init_norm = init_norm
        self.normalize = normalize

        super(GeM, self).__init__(**kwargs)

    def build(self, input_shape):
        feature_size = input_shape[-1]
        self.p = self.add_weight(name="norms", shape=(feature_size,),
                                 initializer=keras.initializers.constant(self.init_norm),
                                 trainable=True)
        super(GeM, self).build(input_shape)

    def call(self, inputs):
        x = inputs
        x = tf.math.maximum(x, 1e-6)
        x = tf.pow(x, self.p)

        
        x=GlobalAveragePooling2D()(x)
        x = tf.pow(x, 1.0 / self.p)

        return x


    def compute_output_shape(self, input_shape):
        return tuple([None, input_shape[-1]])
def gc_att(inp):
    _,h,w,c=inp.shape
    x=Conv2D(1,1,1)(inp)
    x=Reshape((-1,1))(x)
    x=Activation("softmax")(x)#hw,1
    y=Reshape((-1,h*w))(inp)#c, hw
    x=K.batch_dot(y,x)
    x=Reshape((1,1,-1))(x)#c, hw
    x=Conv2D(c//8,1,1)(x)
    x=LayerNormalization()(x)
    x=Activation("relu")(x)
    x=Conv2D(c,1,1)(x)
    return x+inp
def load_model():   
  K.clear_session() 
  mod=DenseNet121(include_top=True, weights='imagenet')
  x = mod.layers[-3].output
  p=self_attention(x,"1")
  q=self_attention(x,"2")
  r=self_attention(x,"3")
  s=self_attention(x,"4")
  x=Concatenate()([p,q,r,s])
  x=Conv2D(1024,1,1)(x)
  x=GeM()(x)
  x=Dense(3,activation='softmax')(x)
  model = Model(inputs=mod.input, outputs=x)
  
  return model